In [1]:
from pandas import Series, DataFrame
import pandas as pd
#from shapely.geometry import Point
#import geopandas as gpd
#from geopandas import GeoDataFrame
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('bmh')
from patsy import dmatrices
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn import tree

In [2]:
#Reading the dataset csv file to create a dataframe
telco = pd.read_csv('Telco_customer_churn.csv', dtype = 'str')

#Data cleaning
telco["Total Charges"].replace({" ": "0", " ": "0"}, inplace=True)

##Removing unnecessary columns
telco = telco.drop(telco.columns[[0,1,2,3,4,5,6,7,8,28,30]], axis=1)
#telco['Latitude'] = telco['Latitude'].astype(float)
#telco['Longitude'] = telco['Longitude'].astype(float)
telco['Churn Value'] = telco['Churn Value'].astype(int)
telco['Tenure Months'] = telco['Tenure Months'].astype(int)
telco['Monthly Charges'] = telco['Monthly Charges'].astype(float)
telco['Total Charges'] = telco['Total Charges'].astype(float)
telco['CLTV'] = telco['CLTV'].astype(int)
telco

,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,...,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Value,CLTV,Churn Reason
0,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1,3239,Competitor made better offer
1,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,...,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1,2701,Moved
2,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.50,1,5372,Moved
3,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,...,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,1,5003,Moved
4,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,...,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.30,1,5340,Competitor had better devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,Female,No,No,No,72,Yes,No,No,No internet service,No internet service,...,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,1419.40,0,5306,NaN
7039,Male,No,Yes,Yes,24,Yes,Yes,DSL,Yes,No,...,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,0,2140,NaN
7040,Female,No,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,...,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,0,5560,NaN
7041,Female,No,Yes,Yes,11,No,No phone service,DSL,Yes,No,...,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,0,2793,NaN


In [3]:
telco = pd.get_dummies(telco, columns = ['Paperless Billing','Gender','Senior Citizen','Partner','Dependents', 'Phone Service', 'Multiple Lines',\
                                         'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', \
                                         'Tech Support', 'Streaming TV', 'Streaming Movies'])
telco.drop(['Paperless Billing_No','Gender_Male','Senior Citizen_No','Partner_No','Dependents_No', 'Multiple Lines_No', 
            'Multiple Lines_No phone service', 'Internet Service_Fiber optic', 'Internet Service_No', 'Online Security_No', 
            'Online Security_No internet service', 'Online Backup_No', 'Online Backup_No internet service', 'Phone Service_No',
            'Device Protection_No', 'Device Protection_No internet service', 'Tech Support_No', 'Tech Support_No internet service', 'Streaming TV_No', 
            'Streaming TV_No internet service', 'Streaming Movies_No', 'Streaming Movies_No internet service'], axis = 1, \
             inplace = True)
telco.columns.values

array(['Tenure Months', 'Contract', 'Payment Method', 'Monthly Charges',
       'Total Charges', 'Churn Value', 'CLTV', 'Churn Reason',
       'Paperless Billing_Yes', 'Gender_Female', 'Senior Citizen_Yes',
       'Partner_Yes', 'Dependents_Yes', 'Phone Service_Yes',
       'Multiple Lines_Yes', 'Internet Service_DSL',
       'Online Security_Yes', 'Online Backup_Yes',
       'Device Protection_Yes', 'Tech Support_Yes', 'Streaming TV_Yes',
       'Streaming Movies_Yes'], dtype=object)

In [4]:
telco["Cross_Product_Count"] = telco["Phone Service_Yes"] + telco["Multiple Lines_Yes"] + telco["Internet Service_DSL"] + \
                               telco["Online Security_Yes"] + telco["Online Backup_Yes"] + telco["Device Protection_Yes"] + \
                               telco["Tech Support_Yes"] + telco["Streaming TV_Yes"] + telco["Streaming Movies_Yes"]

telco["Online_Payment"] = (telco["Payment Method"] == "Mailed check").astype(int)
telco["Contract_Flag"] = (telco["Contract"] != "Month-to-month").astype(int)
#telco['Paperless Billing'] = telco['Paperless Billing'].astype(int)



telco.columns = telco.columns.str.replace(' ','_')
telco.columns.values


array(['Tenure_Months', 'Contract', 'Payment_Method', 'Monthly_Charges',
       'Total_Charges', 'Churn_Value', 'CLTV', 'Churn_Reason',
       'Paperless_Billing_Yes', 'Gender_Female', 'Senior_Citizen_Yes',
       'Partner_Yes', 'Dependents_Yes', 'Phone_Service_Yes',
       'Multiple_Lines_Yes', 'Internet_Service_DSL',
       'Online_Security_Yes', 'Online_Backup_Yes',
       'Device_Protection_Yes', 'Tech_Support_Yes', 'Streaming_TV_Yes',
       'Streaming_Movies_Yes', 'Cross_Product_Count', 'Online_Payment',
       'Contract_Flag'], dtype=object)

In [5]:
Y, X = dmatrices('Churn_Value ~ 0 + Tenure_Months + Paperless_Billing_Yes + Monthly_Charges +Total_Charges + CLTV + Gender_Female \
                 +Senior_Citizen_Yes+Partner_Yes+Dependents_Yes+Phone_Service_Yes+Multiple_Lines_Yes +Internet_Service_DSL+\
                  Online_Security_Yes+Online_Backup_Yes+Device_Protection_Yes+ Tech_Support_Yes+ Streaming_TV_Yes+ \
                 Streaming_Movies_Yes+Cross_Product_Count+Online_Payment+Contract_Flag', telco , return_type='dataframe')
y = Y['Churn_Value'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6)
model.fit(X_train, y_train)

from sklearn import metrics

prediction_train = model.predict(X_train)
print(metrics.accuracy_score(y_train, prediction_train))

prediction = model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))

print('Precision: ',metrics.precision_score(y_test, prediction))
print('Recall ', metrics.recall_score(y_test,prediction))

0.8010223400227187
0.7876206700738216
Precision:  0.5705645161290323
Recall  0.6373873873873874


In [6]:
X

,Tenure_Months,Paperless_Billing_Yes,Monthly_Charges,Total_Charges,CLTV,Gender_Female,Senior_Citizen_Yes,Partner_Yes,Dependents_Yes,Phone_Service_Yes,...,Internet_Service_DSL,Online_Security_Yes,Online_Backup_Yes,Device_Protection_Yes,Tech_Support_Yes,Streaming_TV_Yes,Streaming_Movies_Yes,Cross_Product_Count,Online_Payment,Contract_Flag
0,2.0,1.0,53.85,108.15,3239.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0
1,2.0,1.0,70.70,151.65,2701.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,8.0,1.0,99.65,820.50,5372.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,5.0,0.0,0.0
3,28.0,1.0,104.80,3046.05,5003.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,6.0,0.0,0.0
4,49.0,1.0,103.70,5036.30,5340.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,72.0,1.0,21.15,1419.40,5306.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
7039,24.0,1.0,84.80,1990.50,2140.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,8.0,1.0,1.0
7040,72.0,1.0,103.20,7362.90,5560.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,1.0,6.0,0.0,1.0
7041,11.0,1.0,29.60,346.45,2793.0,1.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [7]:
importance = model.feature_importances_
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.16069
Feature: 1, Score: 0.00334
Feature: 2, Score: 0.16667
Feature: 3, Score: 0.02994
Feature: 4, Score: 0.01763
Feature: 5, Score: 0.00367
Feature: 6, Score: 0.00570
Feature: 7, Score: 0.00000
Feature: 8, Score: 0.10710
Feature: 9, Score: 0.00777
Feature: 10, Score: 0.00000
Feature: 11, Score: 0.01252
Feature: 12, Score: 0.00518
Feature: 13, Score: 0.00000
Feature: 14, Score: 0.00000
Feature: 15, Score: 0.01398
Feature: 16, Score: 0.00000
Feature: 17, Score: 0.00000
Feature: 18, Score: 0.00000
Feature: 19, Score: 0.00000
Feature: 20, Score: 0.46581


## Precision and recall

In [8]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

[[1104  213]
 [ 161  283]]
              precision    recall  f1-score   support

         0.0       0.87      0.84      0.86      1317
         1.0       0.57      0.64      0.60       444

    accuracy                           0.79      1761
   macro avg       0.72      0.74      0.73      1761
weighted avg       0.80      0.79      0.79      1761



# variables removed

Phone Service   ,Multiple Lines    Internet Service    Online Security    Online Backup    Device Protection    Tech Support    Streaming TV    Streaming Movies

# running the model after removing variables

In [9]:
Y, X = dmatrices('Churn_Value ~ 0 + Tenure_Months + Paperless_Billing_Yes + Monthly_Charges +Total_Charges + CLTV + Gender_Female \
                 +Senior_Citizen_Yes+Partner_Yes+Dependents_Yes++Cross_Product_Count+Online_Payment+Contract_Flag', telco , return_type='dataframe')
y = Y['Churn_Value'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=6)
model.fit(X_train, y_train)

from sklearn import metrics

prediction_train = model.predict(X_train)
print(metrics.accuracy_score(y_train, prediction_train))

prediction = model.predict(X_test)
print(metrics.accuracy_score(y_test, prediction))


print('Precision: ',metrics.precision_score(y_test, prediction))
print('Recall ', metrics.recall_score(y_test,prediction))

0.8025369178341537
0.7819420783645656
Precision:  0.575
Recall  0.5180180180180181


In [10]:
X

,Tenure_Months,Paperless_Billing_Yes,Monthly_Charges,Total_Charges,CLTV,Gender_Female,Senior_Citizen_Yes,Partner_Yes,Dependents_Yes,Cross_Product_Count,Online_Payment,Contract_Flag
0,2.0,1.0,53.85,108.15,3239.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0
1,2.0,1.0,70.70,151.65,2701.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
2,8.0,1.0,99.65,820.50,5372.0,1.0,0.0,0.0,1.0,5.0,0.0,0.0
3,28.0,1.0,104.80,3046.05,5003.0,1.0,0.0,1.0,1.0,6.0,0.0,0.0
4,49.0,1.0,103.70,5036.30,5340.0,0.0,0.0,0.0,1.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7038,72.0,1.0,21.15,1419.40,5306.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
7039,24.0,1.0,84.80,1990.50,2140.0,0.0,0.0,1.0,1.0,8.0,1.0,1.0
7040,72.0,1.0,103.20,7362.90,5560.0,1.0,0.0,1.0,1.0,6.0,0.0,1.0
7041,11.0,1.0,29.60,346.45,2793.0,1.0,0.0,1.0,1.0,2.0,0.0,0.0


In [11]:
pip install graphviz

In [12]:
conda install python-graphviz

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\saite\Anaconda3

  added / updated specs:
    - python-graphviz


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              conda-forge::conda-4.10.3-py38haa244f~ --> pkgs/main::conda-4.10.3-py38haa95532_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [13]:
conda install -c conda-forge pydotplus

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\saite\Anaconda3

  added / updated specs:
    - pydotplus


The following packages will be SUPERSEDED by a higher-priority channel:

  conda              pkgs/main::conda-4.10.3-py38haa95532_0 --> conda-forge::conda-4.10.3-py38haa244fe_0


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [17]:
import pydotplus
import graphviz
# DOT data
dot_data = tree.export_graphviz(model, out_file=None, filled=True, rounded=True, \
                                special_characters=True, feature_names= X.columns.values, class_names=['0','1'])

# Draw graph
#graph = graphviz.Source(dot_data, format="png") 
#graph


pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.write_png('original_tree_2.png')
pydot_graph.set_size('"5,5!"')
pydot_graph.write_png('resized_tree_2.png')

True

In [15]:
importance = model.feature_importances_
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.15754
Feature: 1, Score: 0.00339
Feature: 2, Score: 0.18338
Feature: 3, Score: 0.03682
Feature: 4, Score: 0.02709
Feature: 5, Score: 0.00373
Feature: 6, Score: 0.00579
Feature: 7, Score: 0.00219
Feature: 8, Score: 0.10719
Feature: 9, Score: 0.00000
Feature: 10, Score: 0.00000
Feature: 11, Score: 0.47288


In [16]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, prediction))
print(classification_report(y_test, prediction))

[[1147  170]
 [ 214  230]]
              precision    recall  f1-score   support

         0.0       0.84      0.87      0.86      1317
         1.0       0.57      0.52      0.55       444

    accuracy                           0.78      1761
   macro avg       0.71      0.69      0.70      1761
weighted avg       0.78      0.78      0.78      1761

